# TM10007 Group 4 -- ECG Data

Hallo tekst

In [4]:
# Loading packages
import zipfile
import os
import pandas as pd
import sklearn as sk
import numpy as np



## 1. Data loading, cleaning and visualization

Below we load the ECG data, convert it to a dataframe an clean the data.First we are going to start with loading the CS file into a dataframe

In [2]:
# Load CSV file into dataframe
!git clone https://github.com/JosefienBerg/TM10007_ML_ECG_group4.git

with zipfile.ZipFile('/content/TM10007_ML_ECG_group4/ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/TM10007_ML_ECG_group4/ecg')

data = pd.read_csv('/content/TM10007_ML_ECG_group4/ecg/ecg_data.csv', index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
print(data)

Cloning into 'TM10007_ML_ECG_group4'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 159 (delta 47), reused 54 (delta 36), pack-reused 82
Receiving objects: 100% (159/159), 67.95 MiB | 31.77 MiB/s, done.
Resolving deltas: 100% (71/71), done.
The number of samples: 827
The number of columns: 9001
            0_0         0_1         0_2        0_3        0_4         0_5  \
0     47.685046   48.416904   37.371002  61.604757  58.732738  213.067966   
1    152.726718  297.353726  111.168880  25.419423  14.636085   24.609688   
2      1.601260    3.882169   22.978997  21.673040   8.919484   18.329286   
3      1.388947    3.052483    3.084103   4.627886  10.016196   15.020347   
4      3.625561    3.728466    6.205367  17.722897   5.699401    9.024836   
..          ...         ...         ...        ...        ...         ...   
822   14.673713   13.879148   10.000083   8.337630   3.55730

Now we are going to test if there is data missing in our dataset. We conclude that NaNs are missing data and if an entire row or column has zero's, this is also seen as missing data.

In [ ]:
# Checking if there are any NaN or None values in the dataset
print(f'Are there any NaN or None values in the dataset? = '+ str(data.isnull().values.any()))

Are there any NaN or None values in the dataset? = False


In [38]:
# Test dataset for missing rows/columns

# Columns
columnswithzeros = []
for (colname, colval) in data.iteritems():
  if (colval.to_numpy()==0).all() == True:
    print(f'All values are zero in column {colname}')
    columnswithzeros.append(colname)

if len(columnswithzeros) == 0:
  print('There are no columns with only zeros')
  
# Rows
rowswithzeros = []
for (index, rowval) in data.iterrows():
  if (rowval.to_numpy()==0).all() == True: 
    print(f'All values are zero in row {index}')
    rowswithzeros.append(index)

if len(rowswithzeros) == 0:
  print('There are no rows with only zeros')

There are no columns with only zeros
There are no rows with only zeros


Lasty we are going to check if there are patiens or features with multiple zeros in order to see if those zero's could represent missing data

In [101]:
# Test patients with multiple zero's 
def check_for_zeros_in_patients(data): 
  patients_with_zeros=[]
  for (index, rowval) in data.iterrows():
    row = rowval.to_numpy()[:-1] #change to numpy to delete row names en the last row of labels
    if (row==0).any() == True: #if there is a zero in the row
      number_of_zeros = 0
      for i in range(len(row)): #check the entire length of the row for
        if row[i] == 0:
          number_of_zeros += 1
      patients_with_zeros.append({"patient number": index, "number of zeros": number_of_zeros}) 
  print(patients_with_zeros)
  print(f' There are {len(patients_with_zeros)} patients with zeros')

check_for_zeros_in_patients(data)

# Test features with multiple zero's
def check_for_zeros_in_features(data):
  features_with_zeros = []
  for (colname, colval) in data.iteritems():
    if colname == "label": #don't check the column label
      continue
    col = colval.to_numpy()
    if (col==0).any() == True: 
      number_of_zeros = 0
      for i in range(len(col)):
        if col[i] == 0:
          number_of_zeros += 1
      features_with_zeros.append({"feature": colname, "number of zeros": number_of_zeros}) 

  print(features_with_zeros)
  print(f'There are {len(features_with_zeros)} features with zeros')

check_for_zeros_in_features(data)

[{'patient number': 177, 'number of zeros': 750}, {'patient number': 251, 'number of zeros': 750}, {'patient number': 269, 'number of zeros': 750}, {'patient number': 321, 'number of zeros': 750}, {'patient number': 323, 'number of zeros': 750}, {'patient number': 385, 'number of zeros': 750}, {'patient number': 434, 'number of zeros': 750}, {'patient number': 446, 'number of zeros': 750}, {'patient number': 537, 'number of zeros': 750}, {'patient number': 542, 'number of zeros': 750}, {'patient number': 575, 'number of zeros': 750}, {'patient number': 601, 'number of zeros': 750}, {'patient number': 784, 'number of zeros': 750}, {'patient number': 790, 'number of zeros': 750}]
 There are 14 patients with zeros
[{'feature': '6_0', 'number of zeros': 1}, {'feature': '6_1', 'number of zeros': 1}, {'feature': '6_2', 'number of zeros': 1}, {'feature': '6_3', 'number of zeros': 1}, {'feature': '6_4', 'number of zeros': 1}, {'feature': '6_5', 'number of zeros': 1}, {'feature': '6_6', 'number

Conclusion: there are patients with multiple zero's (14 with 750 zero's), however these are not centered at certain features. This means that these zero's could indeed represent missing data and are therefore be replaced with the mean of that feature for the label of that patient.


In [102]:
# Create a list of means per label and feature
data_normal = data[data["label"]== 0]
data_abnormal = data[data["label"]== 1]
data_normal_mean = {}
data_abnormal_mean = {}

for (colname, colval) in data_normal.iteritems():
  if colname == "label":
    continue
  data_normal_mean[colname] = (colval.to_numpy().mean())

for (colname, colval) in data_abnormal.iteritems():
   if colname == "label":
     continue
   data_abnormal_mean[colname] = (colval.to_numpy().mean())

print(data_normal_mean)
print(data_abnormal_mean)

{'0_0': 469.3470502749906, '0_1': 70.6996088100883, '0_2': 40.62862319412772, '0_3': 31.173712757341196, '0_4': 31.54865255065674, '0_5': 43.429041907720126, '0_6': 55.84330178557533, '0_7': 39.003234790730296, '0_8': 25.226056527835073, '0_9': 19.172557363950904, '0_10': 22.256479757512093, '0_11': 26.297794080493855, '0_12': 30.72142554697234, '0_13': 31.66812879477023, '0_14': 30.372431869497436, '0_15': 31.39360833307335, '0_16': 33.352713458528626, '0_17': 34.58298517061866, '0_18': 38.503790736701596, '0_19': 38.319370868571966, '0_20': 43.68758295776539, '0_21': 35.40656152935558, '0_22': 34.785388992521355, '0_23': 35.78567016939342, '0_24': 34.731802677249625, '0_25': 30.919289833729547, '0_26': 28.888807336884312, '0_27': 25.994899925065507, '0_28': 25.159825027510696, '0_29': 25.99326216312191, '0_30': 28.119506596374972, '0_31': 30.27387223375175, '0_32': 31.95308050038741, '0_33': 30.74525789346074, '0_34': 31.30504561055703, '0_35': 31.8473466118405, '0_36': 32.1852669540

In [104]:
# Replace zeros with the correct mean of that feature and label
cleaned_data = data.copy().to_numpy() #make a copy of original data and change into numpy array

column_names = list(data.columns.values) #make a list of the column names

for i in range(len(cleaned_data)):
  for j in range(len(cleaned_data[0])-1):
    if cleaned_data[i,j] == 0:
      label = cleaned_data[i,-1]
      if label == 0:
        col_name = column_names[j]
        cleaned_data[i,j] = data_normal_mean[col_name]
      else:
        col_name = column_names[j]
        cleaned_data[i,j] = data_abnormal_mean[col_name]

# Change back from numpy array to dataframe and check if there are no zeros left and the dimension are the same
cleaned_df = pd.DataFrame(cleaned_data, columns = column_names)
print(f'The number of samples: {len(cleaned_df.index)}')
print(f'The number of columns: {len(cleaned_df.columns)}')
check_for_zeros_in_patients(cleaned_df)
check_for_zeros_in_features(cleaned_df)

The number of samples: 827
The number of columns: 9001
[]
 There are 0 patients with zeros
[]
There are 0 features with zeros


Lastly we want to categorize our dataset to see how many patients have a normal and how many patients have an abnormal ECG. 


In [105]:
# How many patients have a normal or abnormal ECG?
count0= (cleaned_df['label'] == 0).sum()
count1= (cleaned_df['label'] == 1).sum()
print(f'There are {count0} patients with label 0')
print(f'There are {count1} patients with label 1')

# Since the majority of patients has no abnormalities we can conclude that label 0 is normal and label 1 is abnormal

There are 681 patients with label 0
There are 146 patients with label 1


## 2. Creating the experimental setup
Below we split the dataset in test data and training data. Then we further split the trainingdata in order to make different training and validation sets for our different classifiers

In [ ]:
# Initial split

# packages 
from sklearn.model_selection import train_test_split

def train_test_split(x, y, train_size = 0.8, random_state = 0):
  return x_train, x_test, y_train, y_test 

col = "label"
x_train, x_test, y_train, y_test = train_test_split (data.loc[:, data.columns !=col], data.loc[:, "label"].values) 

print(y_train)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 1
 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 1 1 1 1 0 0 1 0 0 1 0 1 0 0 1 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0
 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 1 0 0 0 0 0 0 0 0 0 

hallo text

In [ ]:
# Cross-validatie

hallo text

In [ ]:
# Extra blokje

## 3. Feature scaling

Hallo tekst

In [ ]:
# Blokje 1

Hallo tekst

In [ ]:
# Blokje 2

Hallo tekst

In [ ]:
# Blokje 3

## 4. Feature selection

Hallo tekst

In [ ]:
# Blokje 1

Hallo tekst

In [ ]:
# Blokje 2

Hallo tekst

In [ ]:
# Blokje 3